# ISA-01

* This notebook is used to preprocess item-pool so that each item has word segmentation, answer option index and their target sense available.
* The item pool is based on all_questions_with_ratings.json and WSD_all_questions.xlsx

In [1]:
import wasp
import numpy as np
import pandas as pd

In [2]:
item_pool_dir = wasp.get_data_path("item_pool", "")

In [3]:
import json
import jieba
jieba.add_word("小華")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\seantyh\AppData\Local\Temp\jieba.cache
Loading model cost 0.660 seconds.
Prefix dict has been built succesfully.


In [4]:
raw_all = pd.read_excel(wasp.get_data_path("ratings", "WSD_all_questions.xlsx"), index_col=0)
raw_all["question_stem"] = raw_all["question_stem"].str.replace("\n", " ")
raw_all = raw_all.set_index("question_stem")

In [5]:
from tqdm.autonotebook import tqdm

with open(wasp.get_data_path("item_pool", "all_questions_with_ratings.json"), "r", encoding="UTF-8") as fin:
    items = json.load(fin)
    
for item_x in items:
    df = raw_all.loc[item_x["question"], :].reset_index(drop=True)        
    ans_index = np.where(df.is_Answer==1)[0][0]
    target_sense = df["target_sense"].tolist()[0]
    item_x["target_sense"] = target_sense
    item_x["ans_index"] = int(ans_index)

c:\python37\lib\site-packages\ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [6]:
items[0]

{'qid': 1,
 'MaxIsCorrect': True,
 'RatingMax': 4.2,
 'Rating2nd': 1.6,
 'max_2nd': 2.625,
 'XbarSD': 1.4095,
 'question': '小華翹班偷偷去<會>情人，被他老闆抓個正著。',
 'options': ['小華有見到情人。',
  '小華用情人當話題去交朋友。',
  '小華當情人的能力極佳，大家都想跟他學習。',
  '小華有可能成為你的情人。'],
 'options_rating': [4.2, 1.2, 1.4, 1.6],
 'options_ratingSD': [0.8367, 0.4472, 0.8944, 1.3416],
 'target_sense': '面對面相見。',
 'ans_index': 0}

In [7]:
def segment(x):
    x["question"], x["options"] = wasp.segment_words(x["question"], x["options"])
    return x
    
item_iter = map(segment, items)
items = list(tqdm(item_iter, total=len(items)))

In [8]:
with open(wasp.get_data_path("item_pool", "item_pool_supp.json"), "w", encoding="UTF-8") as fout:
    json.dump(items, fout, ensure_ascii=False, indent=2)